In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from utils.func_utils import accept, jacobian, autocovariance, get_log_likelihood, get_data, binarize, normal_kl
from utils.distributions import Gaussian, GMM, GaussianFunnel, gen_ring
from utils.layers import Linear
from utils.dynamics import Dynamics

In [23]:
tf.reset_default_graph()

In [24]:
size1 = 10
size2 = 10

class Network(object):
    def __init__(self, x_dim, scope='Network', factor=1.0):
        with tf.variable_scope(scope):
            self.embed_1 = Linear(x_dim, size1, scope='embed_1', factor=1.0 / 3)
            self.embed_2 = Linear(x_dim, size1, scope='embed_2', factor=factor * 1.0 / 3)
            self.embed_3 = Linear(2, size1, scope='embed_3', factor=1.0 / 3)

            self.linear_1 = Linear(size1, size2, scope='linear_1')

            self.scaling_S = tf.exp(tf.get_variable('scale_s', shape=(1, x_dim), initializer=tf.constant_initializer(0.)))
            self.scaling_F = tf.exp(tf.get_variable('scale_f', shape=(1, x_dim), initializer=tf.constant_initializer(0.)))      
            self.linear_s = Linear(size2, x_dim, scope='linear_s', factor=0.001)
            self.linear_t = Linear(size2, x_dim, scope='linear_t', factor=0.001)
            self.linear_f = Linear(size2, x_dim, scope='linear_f', factor=0.001)
      
    def hidden(self, x, v, t):
        z1 = self.embed_1(x)
        z2 = self.embed_2(v)
        z3 = self.embed_3(t)

        h1 = tf.nn.relu(z1 + z2 + z3)

        h2 = tf.nn.relu(self.linear_1(h1))
        # h3 = tf.nn.relu(self.linear_2(h2))
        return tf.nn.relu(h2)

    def S(self, x, v, t, aux=None):
        h = self.hidden(x, v, t)
        use_tanh = True
        if use_tanh:
            return self.scaling_S * tf.nn.tanh(self.linear_s(h))
        else: 
            return self.linear_s(h)

    def T(self, x, v, t, aux=None):
        h = self.hidden(x, v, t)
        return self.linear_t(h)

    def F(self, x, v, t, aux=None):
        h = self.hidden(x, v, t)
        return self.scaling_F * tf.nn.tanh(self.linear_f(h))

def net_factory(x_dim, scope, factor):
    return Network(x_dim, scope=scope, factor=factor)

In [25]:
def boolean_mask(x, mask):
    return tf.boolean_mask(x, mask), tf.boolean_mask(x, tf.logical_not(mask))

def inverse_boolean_mask(x_true, x_false, mask):
    n = tf.shape(mask)[0]
    rest = x_true.get_shape().as_list()[1:]
    shape = [n] + rest

    ind1 = tf.expand_dims(gather_indices(mask), 1)
    ind2 = tf.expand_dims(gather_indices(tf.logical_not(mask)), 1)

    z = tf.scatter_nd(ind1, x_true, shape) + tf.scatter_nd(ind2, x_false, shape)
    return z

def gather_indices(mask):
    n = tf.shape(mask)[0]
    return tf.boolean_mask(tf.range(n), mask)

In [26]:
loss_name = 'inv'
x_dim = 5
lr = tf.placeholder(tf.float32, shape=())
g = Gaussian(np.zeros((5,)), np.diag([100, 0.1, 10, 1.0, 0.01]))
# g = GaussianFunnel(dim=2, clip=12.)
# M = get_rotation(np.pi / 4)
# mu = np.zeros(2,)
# sigma = np.array([[100, 0.0], [0., 0.1]])

# g = Gaussian(mu, M.dot(sigma).dot(M.T))

dynamics = Dynamics(x_dim, g.get_energy_function(), T=5, eps=0.1, hmc=False, eps_trainable=True, net_factory=net_factory, use_temperature=False)

x = tf.placeholder(tf.float32, shape=(None, x_dim))
z = tf.random_uniform(tf.shape(x))

mask = tf.cast(tf.random_uniform((tf.shape(x)[0],), maxval=2, dtype=tf.int32), tf.bool)
x1, x2 = boolean_mask(x, mask)
z1, z2 = boolean_mask(z, mask)

Lx1, _, px1 = dynamics.forward(x1)
Lx2, _, px2 = dynamics.backward(x2)

Lz1, _, pz1 = dynamics.forward(z1)
Lz2, _, pz2 = dynamics.forward(z2)

Lx = inverse_boolean_mask(Lx1, Lx2, mask)
Lz = inverse_boolean_mask(Lz1, Lz2, mask)
px = inverse_boolean_mask(px1, px2, mask)
pz = inverse_boolean_mask(pz1, pz2, mask)

loss = 0.

v1 = (tf.reduce_sum(tf.square(x - Lx), axis=1) * px) + 1e-4
v2 = (tf.reduce_sum(tf.square(z - Lz), axis=1) * pz) + 1e-4
scale = 1.0

loss += scale * (tf.reduce_mean(1.0 / v1) + tf.reduce_mean(1.0 / v2))
loss += (- tf.reduce_mean(v1) - tf.reduce_mean(v2)) / scale

global_step = tf.Variable(0., name='global_step', trainable=False)

learning_rate = tf.train.exponential_decay(lr, global_step,
                                           250, 0.96, staircase=True)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

def hack_grads(g, w):
    return tf.where(tf.is_finite(g), g, w)

train_op = optimizer.minimize(loss, global_step=global_step)

no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux
no aux


In [ ]:
nb_steps = 10000

noise_to_samples = []
sample_list = []

samples = np.random.randn(n_samples, x_dim)
sample_list.append(np.copy(samples))

sample_acc_prob = []
noise_acc_prob = []

loss_values = []
numbers_list = []

sess = tf.Session()
sess.run(tf.global_variables_initializer())

temp = 1.0

for t in range(nb_steps):
#   if 0 <= t < 1000:
#     temp = 5.
#   elif 1000 <= t < 2000:
#     temp = 5. / 2
#   elif 2000 <= t < 3000:
#     temp = 5. / 4
#   else:
#     temp = 1.0

  if t % 100 == 0:
    temp *= 0.9
  
  if temp <= 1.0:
    temp = 1.0
  
  mask = np.random.randint(low=0, high=2, size=(n_samples,))
  samples_forward = samples[mask == 0]
  samples_backward = samples[mask == 1]

  feed_dict={
      x_to_forward: samples_forward,
      x_to_backward: samples_backward,
      s.temperature: temp,
      lr: 1e-3,
  }

  _, loss_, x_prev, x_prop, p_x_, z_prev, z_prop, p_z_, numbers_ = sess.run([
      train_op,
      loss,
      x_previous,
      x_proposed,
      p_x,
      z_previous,
      z_proposed,
      p_z,
      numbers
  ], feed_dict)
  
  numbers_list.append(numbers_)
  sample_acc_prob.append(np.mean(p_x_))
  noise_acc_prob.append(np.mean(p_z_))

  samples = accept(x_prev, x_prop, p_x_)
  sample_list.append(np.copy(samples))
  noise_to_sample = accept(z_prev, z_prop, p_z_)
  noise_to_samples.append(np.copy(noise_to_sample))
  loss_values.append(loss_)
  
  if t % 100 == 0:
    print 'Step: %d / %d, Loss: %.2e, Acceptance sample: %.2f, Acceptance Noise: %.2f, Epsilon: %.3f, LR: %.5f' % (t, nb_steps, loss_, np.mean(p_x_), np.mean(p_z_), sess.run(s.eps), sess.run(learning_rate, {lr: 1e-3}))

In [ ]:
  # post training sampling
  
  # samples = g.get_samples(n=n_samples)
  samples = np.random.randn(n_samples, 2)
  final_samples = []
  
  for t in range(200):
    final_samples.append(np.copy(samples))
    mask = np.random.randint(low=0, high=2, size=(n_samples,))
    samples_forward = samples[mask == 0]
    samples_backward = samples[mask == 1]
    
    feed_dict = {
        x_to_forward: samples_forward,
        x_to_backward: samples_backward,
        s.temperature: 1.,
    }
    
    x_f_prop, x_b_prop, p_forward, p_backward = sess.run([x_forward_proposed, x_backward_proposed, p_x_forward, p_x_backward], feed_dict)
        
    new_samples = np.zeros((n_samples, x_dim))
    p_accept = np.zeros((n_samples,))
    
    new_samples[mask == 0] = x_f_prop
    new_samples[mask == 1] = x_b_prop
    
    p_accept[mask == 0] = p_forward
    p_accept[mask == 1] = p_backward
    
    samples = accept(samples, new_samples, p_accept)